In [1]:
print('hi')

hi


In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Bhavya Prakash\\OneDrive\\Desktop\\Text summarizer\\Text-Summarizer\\research'

In [4]:
os.chdir('../')

In [5]:
from dataclasses import dataclass

In [6]:
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL:str
    local_data_file: Path
    unzip_dir: Path
    

In [7]:
from text_summarizer.constants import *
from text_summarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        


In [9]:
import os
import urllib.request as request
#import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size
#import zipfile36 as zipfile
import py7zr

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        # zip_file_path = "artifacts/data_ingestion/data.zip"  # Replace with the actual path to your ZIP file

        
        with py7zr.SevenZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)  # Replace "destination_folder" with the folder where you want to extract the files
        # except py7zr.exceptions.Bad7zFile:
        #     print("The file is not a valid 7z archive.")
        # except FileNotFoundError:
        #     print(f"The file '{zip_file_path}' does not exist.")

        # with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
        #     zip_ref.extractall(unzip_path)
 

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-04 09:13:44,324: INFO: common: yaml file:config\config.yaml loaded successfully]
[2023-09-04 09:13:44,353: INFO: common: yaml file:params.yaml loaded successfully]


[2023-09-04 09:13:44,359: INFO: common: created directory at: artifacts]
[2023-09-04 09:13:44,365: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-04 09:13:44,428: INFO: 1267882926: File already exists of size: ~2875 KB]
